# Baran Experiment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas
import IPython.display

import raha
import datetime

In [3]:
def format_delta(delta: datetime.timedelta):
    hours, remainder = divmod(delta.total_seconds(), 3600)
    minutes, seconds = divmod(remainder, 60)
    
    h = f'{int(hours)}h ' if hours > 0 else ''
    m = f'{int(minutes)}m ' if minutes > 0 else ''
    s = f'{int(seconds)}s'
    
    return h+m+s

def estimate_time_to_finish(times: list, current_run_i: int, total_runs: int):
    deltas = []
    i = 1
    
    while i+1 <= len(times):
        deltas.append(times[i] - times[i-1])
        i += 1
        
    avg = sum(deltas, datetime.timedelta())/len(deltas)
    return f'Run {current_run_i}/{total_runs}. {format_delta(avg)} per run, estimate {format_delta(avg*(total_runs - current_run_i))} to finish'

In [4]:
results = []

error_rates = range(1,6)
versions = range(1,6)
datasets = ['bridges']

duration_experiment = len(datasets) * len(error_rates) * len(versions)
times = [datetime.datetime.now()]
i = 1


for dataset in datasets:
    for error_rate_pct in error_rates:
        for version in versions:
            data_dict = {"name": dataset,
                         "path": "../datasets/renuver/{0}/{0}_{1}_{2}.csv".format(dataset, error_rate_pct, version),
                         "clean_path": "../datasets/renuver/{0}/clean.csv".format(dataset)}

            data = raha.Dataset(data_dict)
            data.detected_cells = dict(data.get_actual_errors_dictionary())
            app = raha.Correction()
            app.LABELING_BUDGET = 20
            app.VERBOSE = False
            app.FEATURE_GENERATORS = ['vicinity']
            app.CLASSIFICATION_MODEL = "ABC"
            app.VICINITY_ORDERS = [1, 2]
            app.VICINITY_FEATURE_GENERATOR = 'pdep'
            app.N_BEST_PDEPS = 5
            app.GPDEP_CORRECTION_SCORE_THRESHOLD = 0


            d = app.initialize_dataset(data)
            app.initialize_models(d)
            while len(d.labeled_tuples) < app.LABELING_BUDGET:
                app.sample_tuple(d, random_seed=None)
                app.label_with_ground_truth(d)
                app.update_models(d)
                app.generate_features(d, synchronous=True)
                app.predict_corrections(d, random_seed=None)

                p, r, f = d.get_data_cleaning_evaluation(d.corrected_cells)[-3:]
                result = {'dataset': dataset, 
                          'error_rate_pct': error_rate_pct,
                          'dataset_version': version,
                          'n_sample': len(d.labeled_tuples),
                          'precision': p, 'recall': r, 'f1': f}                    
                results.append(result)
            
            # time estimates for the measurement
            times.append(datetime.datetime.now())
            print(estimate_time_to_finish(times, i, duration_experiment))
            i += 1

Run 1/25. 10s per run, estimate 4m 1s to finish
Run 2/25. 10s per run, estimate 3m 59s to finish
Run 3/25. 10s per run, estimate 3m 42s to finish
Run 4/25. 9s per run, estimate 3m 26s to finish
Run 5/25. 9s per run, estimate 3m 17s to finish
Run 6/25. 9s per run, estimate 3m 6s to finish
Run 7/25. 9s per run, estimate 2m 55s to finish
Run 8/25. 9s per run, estimate 2m 44s to finish
Run 9/25. 9s per run, estimate 2m 34s to finish
Run 10/25. 9s per run, estimate 2m 24s to finish
Run 11/25. 9s per run, estimate 2m 14s to finish
Run 12/25. 9s per run, estimate 2m 6s to finish
Run 13/25. 9s per run, estimate 1m 58s to finish
Run 14/25. 9s per run, estimate 1m 49s to finish
Run 15/25. 9s per run, estimate 1m 39s to finish
Run 16/25. 10s per run, estimate 1m 30s to finish
Run 17/25. 10s per run, estimate 1m 21s to finish
Run 18/25. 10s per run, estimate 1m 11s to finish
Run 19/25. 10s per run, estimate 1m 2s to finish
Run 20/25. 10s per run, estimate 52s to finish
Run 21/25. 10s per run, esti

In [5]:
print(results)

[{'dataset': 'bridges', 'error_rate_pct': 1, 'dataset_version': 1, 'n_sample': 1, 'precision': 1.0, 'recall': 0.21428571428571427, 'f1': 0.35294117647058826}, {'dataset': 'bridges', 'error_rate_pct': 1, 'dataset_version': 1, 'n_sample': 2, 'precision': 1.0, 'recall': 0.42857142857142855, 'f1': 0.6}, {'dataset': 'bridges', 'error_rate_pct': 1, 'dataset_version': 1, 'n_sample': 3, 'precision': 1.0, 'recall': 0.5714285714285714, 'f1': 0.7272727272727273}, {'dataset': 'bridges', 'error_rate_pct': 1, 'dataset_version': 1, 'n_sample': 4, 'precision': 0.9, 'recall': 0.6428571428571429, 'f1': 0.75}, {'dataset': 'bridges', 'error_rate_pct': 1, 'dataset_version': 1, 'n_sample': 5, 'precision': 0.9090909090909091, 'recall': 0.7142857142857143, 'f1': 0.8}, {'dataset': 'bridges', 'error_rate_pct': 1, 'dataset_version': 1, 'n_sample': 6, 'precision': 0.9166666666666666, 'recall': 0.7857142857142857, 'f1': 0.8461538461538461}, {'dataset': 'bridges', 'error_rate_pct': 1, 'dataset_version': 1, 'n_sampl